<a href="https://colab.research.google.com/github/dominique-nshimyimana/Car-State/blob/master/series_car_state_action_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Car State-Action Prection 
#Colab: TensorFlow and GoogleDrive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
%tensorflow_version 2.x

In [3]:
from google.colab import drive
mounted = '/drive'
drive.mount(mounted)

Mounted at /drive


# NN state Evaluation

In [0]:
import tensorflow as tf
import numpy as np
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import Input, Model
from tensorflow.keras.optimizers.schedules import ExponentialDecay


class NeuralDynamics(Model):

    def __init__(self, input_size, out_size, training=False):

        super(NeuralDynamics, self).__init__()

        self.training = training
        self.out_size = out_size
        if type(input_size) is tuple:
          self.state_size = input_size[0]
          self.action_size = input_size[1]
        else:
          self.state_size = input_size
          self.action_size = out_size

        self.layer1 = Dense(300, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l1(0.1))
        self.layer2 = Dense(300, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l1(0.1))

        self.layer_state1a = Dense(256, activation=tf.nn.relu)

        self.layer_state1 = Dense(128, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l1(0.1))

        self.layer_state1b = Dense(64, activation=tf.nn.relu)
        self.layer_state1c = Dense(32, activation=tf.nn.relu)

        self.layer_state2 = Dense(self.out_size, kernel_regularizer=tf.keras.regularizers.l1(0.1))

        self.dropout = Dropout(rate=0.2)

        self.flatten = Flatten()

    @tf.function
    def call(self, inputs):

        #state, action = inputs
        #if type(self.input_size) is tuple and self.state_size[0] != state.shape[0]:
        #  missing_zeros = self.state_size[0]-state.shape[0]
        #  paddings = tf.constant([[missing_zeros, 0], [0, 0]])
        #  state = tf.pad(state, paddings, "CONSTANT")

        x = inputs  # f.concat((state, action), axis=1)
        x = self.layer1(x)
        if self.training:
            x = self.dropout(x)
        x = self.layer2(x)
        if self.training:
            x = self.dropout(x)
        x = self.flatten(x)

        x = self.layer_state1a(x)

        x = self.layer_state1(x)
        if self.training:
            x = self.dropout(x)

        x = self.layer_state1b(x)
        x = self.layer_state1c(x)
        
        state = self.layer_state2(x)
        return state

# Prepare Data

In [0]:
# Load dat from folder
# CSV form: Dataset
# Timestamp, x, y, yaw, xvel, yvel, omega, accel, brake, steering
def load_states_actions(data_dir, cutoff_beginning = 300, cutoff_end = 1000, norm=True):
  data_files = [join(data_dir, f) for f in listdir(data_dir) if isfile(join(data_dir, f)) and ".directory" not in f]

  xs_states = []
  xs_actions = []
  ys = []

  for f in data_files:
      print(f)
      data = np.loadtxt(f, delimiter=', ', skiprows=1, dtype=np.float32)[cutoff_beginning:-cutoff_end, :]
      x_states = [data[i, 1:-3] for i in range(len(data))]
      x_actions = [data[i, -3:] for i in range(len(data))]
      y = x_states[1:]
      x_states, x_actions = x_states[:-1], x_actions[:-1]
      xs_states += x_states
      xs_actions += x_actions

  xs_states = np.vstack(xs_states)
  xs_actions = np.vstack(xs_actions)

  scaler = MinMaxScaler()
  xs_states = scaler.fit_transform(xs_states)
  xs_actions = scaler.fit_transform(xs_actions)
  return xs_states, xs_actions

# convert an array of values into a dataset matrix
# State(s) at a given time t (or time series until t) and Y is the state at the next time (t + 1).
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		dataX.append(np.squeeze(dataset[i:(i+look_back)])) #dataset[i:(i+look_back), 0]
		dataY.append(np.expand_dims(dataset[i + look_back], axis=0)) #dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

def takeSlice(arr, fr, to, name):
    
    result = arr[:,fr:to,:]
    print(name + ": start at " + str(fr) + " - shape: " + str(result.shape))
    return result

### Load Data

In [6]:
# Load Data
data_dir = "/drive/My Drive/neuronyte_logging"
xs_states, xs_actions = load_states_actions(data_dir=data_dir)

/drive/My Drive/neuronyte_logging/NeuroNyte_1585748620.316643
/drive/My Drive/neuronyte_logging/NeuroNyte_1585745385.311904


### Preparre and Look at Data ...

In [7]:
# Split data
x_states_train, x_states_test, x_actions_train, x_actions_test = train_test_split(xs_states, xs_actions, test_size=0.10)

# How many steps/state in back from t and # How many steps/state in future/forward from t
look_back = 10
look_forward = 1

# reshape into X=t and Y=t+1 for states
x_states_train, y_train = create_dataset(x_states_train, look_back)
x_states_test, y_test = create_dataset(x_states_test, look_back)

# Visualize X=t and Y=t+1
t = 3
print(y_test[t])
print(x_states_test[t+1])
print(y_test.shape, y_train.shape)
XS_shape, XA_shape, y_shape = x_states_train.shape, x_actions_train.shape, y_train.shape
#X = np.array(x_states_train).reshape(XS_shape[0], look_back, XS_shape[-1])
#Y = np.array(y_train).reshape(y_shape[0], look_forward, y_shape[-1])

[[0.5276035  0.45216864 0.5702387  0.555199   0.62312835 0.50016236]]
[[0.77648085 0.81285197 0.09078854 0.29369906 0.16058719 0.5005165 ]
 [0.91787237 0.9681816  0.8643869  0.80523497 0.3466267  0.4992078 ]
 [0.31193125 0.04743177 0.749698   0.96927243 0.5259143  0.4994809 ]
 [0.26879603 0.6833418  0.6272962  0.5756756  0.5737585  0.49997318]
 [0.06498486 0.47037116 0.99659884 0.51514816 0.14767575 0.49993768]
 [0.3878985  0.28852874 0.2597765  0.24750796 0.51562715 0.49997196]
 [0.18227232 0.07203329 0.77134943 0.73060024 0.43897957 0.50014406]
 [0.35233432 0.5868572  0.09986043 0.36935642 0.31492844 0.4993997 ]
 [0.96108305 0.8933344  0.9468675  0.5670629  0.29988182 0.49965787]
 [0.5276035  0.45216864 0.5702387  0.555199   0.62312835 0.50016236]]
(105332, 1, 6) (948071, 1, 6)


# Initialize Model

In [0]:
batchsize = 2
decay = ExponentialDecay(0.0004, 20000, 0.99)
cosineDecay = tf.keras.experimental.CosineDecayRestarts(0.0004, 30, t_mul=2.0, m_mul=1.0, alpha=0.00001)

optimizer = tf.keras.optimizers.Adam(learning_rate=cosineDecay)
model = NeuralDynamics((look_back, 6),out_size=6, training=True)
EPOCHS = 200

# Training

In [0]:
#print(x_states_train.shape, x_actions_train.shape, y_train.shape)
#np.array(y_test).reshape(y_test.shape[0], 1, y_test.shape[-1])
print('X: ', x_states_test.shape, 'Y: ', y_test.shape)

# checkpoint
'''
filepath=join(mounted, "My Drive/NeuralModel/lstm_car_state/weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5")
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
# Fit the model
history = model.fit(X, Y, epochs=EPOCHS, validation_split=0.2, verbose=1, batch_size=20000, callbacks=callbacks_list)
'''
#history = model.fit(X, Y, epochs=EPOCHS, validation_split=0.2, verbose=1, batch_size=batchsize)

model.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())
history = model.fit(x_states_test, y_test, epochs=EPOCHS, validation_split=0.3, verbose=1, batch_size=batchsize, shuffle = False)

model.save(join(mounted, 'My Drive/NeuralModel/lstm_car_state/seriesv1'))

X:  (105332, 10, 6) Y:  (105332, 1, 6)
Epoch 1/200
9217/9217 [==============================] - 65s 7ms/step - loss: 11.7780 - val_loss: 0.5793
Epoch 2/200
9213/9217 [============================>.] - ETA: 0s - loss: 0.3583

# Testing

In [0]:
 xs_shape, xa_shape, yt_shape = x_states_test.shape, x_actions_test.shape, y_test.shape
X = np.array(x_states_test).reshape(xs_shape[0], look_back, xs_shape[-1])
Y = np.array(y_test).reshape(yt_shape[0], look_forward, yt_shape[-1])

print('Test(xs, sa, y): ', x_states_test.shape, x_actions_test.shape, y_test.shape)
#result = model.predict((x_states_test, x_actions_test))
result = model.predict(x_states_test)
print('Result: ', result.shape)

# Visualization of result

In [0]:
print(y_test.shape, result.shape)
xs_states_true = y_test[:,:,0:2] # np.squeeze(x_states_test[:,:,0:2], axis=0)
xs_states_pred = result[:,0:2] #  np.squeeze(result[:,:,0:2], axis=0)
print(xs_states_true.shape, xs_states_pred.shape)
xs_states_true = xs_states_true.reshape(xs_states_true.shape[0], xs_states_true.shape[2])
#xs_states_pred = xs_states_pred.reshape(xs_states_pred.shape[0], xs_states_pred.shape[2])

begin, fin = 85000, 85050 # Test constrain due to visualization limit
xs_states_true = xs_states_true[begin:fin:1]
xs_states_pred = xs_states_pred[begin:fin:1]


fig,ax = plt.subplots(1,1,figsize=(16,16))
ax.plot(*zip(*xs_states_true), 'r')
ax.plot(*zip(*xs_states_pred),'b')
plt.xlabel('x pos')
plt.ylabel('y pos')
plt.suptitle("yy pasition as one feature")
plt.show()